In [1]:
# %matplotlib inline
import os
os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cpu,floatX=float32"
import theano
import theano.tensor as T
import network
import network.normFlow as nf
import config
import utils
import utils.mathT as mathT
import utils.theanoGeneral as utilsT
import numpy as np
import utils.optimizers as optimizers

In [2]:
DIM = 2
SAMPLINGNUM = 10

# defining target model, here is a known gaussian
mean = utilsT.sharedf([2.0,1.0])
varmat = utilsT.sharedf( np.eye(DIM)/2. )
logTarget = mathT.multiNormInit(mean,varmat)

In [3]:
iaf = nf.NormFlowModel(2,4,noisestd=1.)
e = iaf.getNoiseVar(SAMPLINGNUM)
z, logqz = iaf.reparam(e)

In [4]:
logpz = logTarget(z)
cost = T.mean( logqz - logpz ) # KL cost

In [5]:
params = iaf.getParams()
print params

[<TensorType(float32, matrix)>, <TensorType(float32, vector)>, <TensorType(float32, vector)>, <TensorType(float32, matrix)>, <TensorType(float32, vector)>, <TensorType(float32, vector)>, <TensorType(float32, matrix)>, <TensorType(float32, vector)>, <TensorType(float32, vector)>, <TensorType(float32, matrix)>, <TensorType(float32, vector)>, <TensorType(float32, vector)>]


In [6]:
grads = [ T.grad(cost,p,consider_constant=[e]) for p in params ]
opt = optimizers.SGD(lr=0.05,momentum=0.9,decay=0.001)
updates = opt.getUpdates(params,grads)
estep = theano.function(inputs=[],outputs=[cost],updates=updates)